* Create MUSE input files, which include:
    1. MUSE input file: "Consumption*.csv"
    2. MUSE input file: "Technodata.csv" 
    3. MUSE input file: "TechnodataTimeslices.csv" 
    4. MUSE input file: "GlobalCommodities.csv"
    5. MUSE input file: "CommIn.csv and CommOut.csv"
    6. MUSE input file: "Projections.csv"
    7. MUSE input file: "ExistingCapacity.csv" 
    - (8. MUSE input file: "Agent.csv" )
    9. MUSE input file: MUSE_input.txt
    
* Date : November 2024

In [1]:
# autoreload
%load_ext autoreload
%autoreload 2
# save the notebook every 10 second
%autosave 10

Autosaving every 10 seconds


In [2]:
## Import libraries
import pandas as pd
import numpy as np
from pathlib import Path
import re
from difflib import get_close_matches

## Locate the input data file (TIMES) and the output file (MUSE)

In [3]:
#### TIMES data folder
data_folder = Path.cwd().parent / 'Residential' / 'TIMES_data_Residential'

# folder to save output
output_folder = Path.cwd().parent / 'Residential' /'MUSE_Files'

# Check if the folder exists, and create it if it doesn't
output_folder.mkdir(parents=True, exist_ok=True)


## Create 1. "Consumptions*.csv"

In [4]:
# set region
region = 'UK'
# select the scenario that we are interested in (for comsuption projection)
scenario = "REFScenario_DECC-Central" #There is no data for "BASE" scenario after 2010, so we select "REFScenario_DECC-Central" scenario or any other scenarios.

# define the years that we are interested in
years = list(range(2010, 2051, 10)) #[2010, 2020, 2030, 2040, 2050]

# set the base year
base_year = 2010

In [5]:
# (1) Read-in the Demand values.

# First we need the following file from TIMES data set
data_file1 = 'Residential sector end use demands - all scenarios - PJ.xlsx'

# Read the data into a DataFrame
df1 = pd.read_excel(data_folder / data_file1, skiprows=[0]) # skip the first row

# find all the demand/commodity types
demand_types = df1['commodity'].unique()
# print(demand_types)

# use the value from BASE scenario for the year 2010 in the selected scenario
for demand_name in demand_types:
    # print("\n",demand_name)
    df1.loc[(df1['commodity'] == demand_name) & (df1['scenario'] == scenario), base_year] = df1.loc[(df1['commodity'] == demand_name) & (df1['scenario'] == 'BASE'), base_year].values[0]

# select the rows that we are interested in, meaning dropping the BASE scenario in df1.
df1 = df1[df1['scenario']==scenario]

# we do not the attribute and scenario coloumns
df1.drop(columns=['attribute','scenario'], inplace=True) 

# Display the DataFrame
df1.head()

,commodity,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
1,RCE [RES.DEMAND.COOKING.OTHER.KETTLE.MICROWAVE.],19.792337,19.967875,20.143413,20.318951,20.494489,20.670027,20.866139,21.062251,21.258363,...,25.286341,25.447566,25.608790,25.770015,25.931239,26.090982,26.250725,26.410468,26.570210,26.729953
6,RCH [RES.DEMAND.COOKING.HOBS.],27.000000,27.239463,27.478925,27.718388,27.957851,28.197314,28.464842,28.732371,28.999900,...,34.494724,34.714661,34.934598,35.154535,35.374471,35.592387,35.810302,36.028217,36.246133,36.464048
11,RCO [RES.DEMAND.COOKING.OVENS.],24.300000,24.515516,24.731033,24.946549,25.162066,25.377582,25.618358,25.859134,26.099910,...,31.045252,31.243195,31.441138,31.639081,31.837024,32.033148,32.229272,32.425396,32.621519,32.817643
16,REA [RES.DEMAND.CONSUMER-ELECTRONICS.TV.DVD.SE...,45.832374,46.150504,46.468634,46.786764,47.104894,47.423024,47.742631,48.062239,48.381846,...,54.364588,54.574626,54.784663,54.994701,55.204738,55.404856,55.604974,55.805092,56.005210,56.205328
21,RECF [RES.DEMAND.FREEZERS.],12.296386,12.405443,12.514499,12.623556,12.732612,12.841669,12.963507,13.085345,13.207184,...,15.709646,15.809810,15.909974,16.010138,16.110302,16.209545,16.308789,16.408032,16.507276,16.606519


In [6]:
# We need to create a dictionary to store the commodity names used in this df1 as key-value pairs.
# This dictionary will be used in the end (after df1 and df2 are concatenated) to replace the commodity names so that they are compatible with the name used in "GlobalCommodoties.csv".

## Dictionary creation
commodity_dict = {}
for item in df1["commodity"]:
    # Split at the first "[" to separate the key and value parts
    key, value = item.split(" [", 1)
    # Remove the trailing "]" from the value part
    value = value.rstrip("].").replace("DEMAND.", "") # remove "RES.DEMAND." part from the value.
    # Add the key-value pair to the dictionary
    # value = value.replace("DEMAND.", "") # remove "RES.DEMAND." part from the value.
    # Split the string by '.' and filter out any empty parts
    parts = [part for part in value.split('.') if part]
    # Join only the first three parts without adding any trailing dot
    commodity_dict[key] = '.'.join(parts[:3])

commodity_dict


{'RCE': 'RES.COOKING.OTHER',
 'RCH': 'RES.COOKING.HOBS',
 'RCO': 'RES.COOKING.OVENS',
 'REA': 'RES.CONSUMER-ELECTRONICS.TV',
 'RECF': 'RES.FREEZERS',
 'RECL': 'RES.COOLING',
 'RECP': 'RES.COMPUTERS',
 'RECR': 'RES.REFRIGERATORS',
 'REL': 'RES.LIGHTING',
 'REO': 'RES.OTHER',
 'REW': 'RES.WET.APPLIANCES',
 'RHEA': 'RES.SPACE-HEAT.EXISTING-AVERAGE',
 'RHNA': 'RES.SPACE-HEAT.NEW-AVERAGE',
 'RWEA': 'RES.HOT-WATER.EXISTING-AVERAGE',
 'RWNA': 'RES.HOT-WATER.NEW-AVERAGE'}

In [7]:
# We also need to replace the values in the commodity column of df1 so that only the part before the square brackets ([]) remains
# This allows us to match the commodity names with the commodity name used in the 'Residential Sector COM_FR - fraction of demand in each time slice.xlsx', as this two data sets have different name convention for the same commodities.
# We will read this file next as df2 and concatenate the two DataFrames.
df1['commodity'] = df1['commodity'].str.split(' ').str[0]
df1.head()

,commodity,2010,2011,2012,2013,2014,2015,2016,2017,2018,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
1,RCE,19.792337,19.967875,20.143413,20.318951,20.494489,20.670027,20.866139,21.062251,21.258363,...,25.286341,25.447566,25.608790,25.770015,25.931239,26.090982,26.250725,26.410468,26.570210,26.729953
6,RCH,27.000000,27.239463,27.478925,27.718388,27.957851,28.197314,28.464842,28.732371,28.999900,...,34.494724,34.714661,34.934598,35.154535,35.374471,35.592387,35.810302,36.028217,36.246133,36.464048
11,RCO,24.300000,24.515516,24.731033,24.946549,25.162066,25.377582,25.618358,25.859134,26.099910,...,31.045252,31.243195,31.441138,31.639081,31.837024,32.033148,32.229272,32.425396,32.621519,32.817643
16,REA,45.832374,46.150504,46.468634,46.786764,47.104894,47.423024,47.742631,48.062239,48.381846,...,54.364588,54.574626,54.784663,54.994701,55.204738,55.404856,55.604974,55.805092,56.005210,56.205328
21,RECF,12.296386,12.405443,12.514499,12.623556,12.732612,12.841669,12.963507,13.085345,13.207184,...,15.709646,15.809810,15.909974,16.010138,16.110302,16.209545,16.308789,16.408032,16.507276,16.606519


In [8]:
# (2) Read-in the "Time-slice" data.

# Then we need the following file from TIMES data set
data_file4 = 'Residential Sector COM_FR - fraction of demand in each time slice.xlsx'
              
# Read the data into a DataFrame df2
df2 = pd.read_excel(data_folder / data_file4, skiprows=[0,1]) # skip the first 2 rows
df2.drop(columns=['attribute','scenario','time_slice','lim_type'], inplace=True) # we do not the attribute coloumn

# transform the DataFrame
df2 = df2.T

# # display the DataFrame
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
RCE,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500
RCH,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500
RCO,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500,0.125,0.025000,0.01250,0.087500
REA,0.125,0.050000,0.03125,0.043750,0.125,0.050000,0.03125,0.043750,0.125,0.050000,0.03125,0.043750,0.125,0.050000,0.03125,0.043750
RECF,0.125,0.035714,0.06250,0.026786,0.125,0.035714,0.06250,0.026786,0.150,0.042857,0.07500,0.032143,0.100,0.028571,0.05000,0.021429


In [9]:
# (3) Use df1 and df2 to create Comsuption*.csv
## set the commodity column as the index, so that we can combine the two DataFrames based on the commodity column
df1.set_index('commodity', inplace=True)

# Ensure the indices are identical and aligned
df1 = df1.sort_index()
df2 = df2.sort_index()


# Multiply each column in df1 with each column in df2
for year in years:
    comsuption = {}
    for demand_name in df2.columns:
        comsuption[f'{demand_name}'] = df1[year] * df2[demand_name]
    
    comsuption_df = pd.DataFrame.from_dict(comsuption, orient='index')

    # replace the column names with the commodity names in commodity_dict
    comsuption_df.rename(columns=commodity_dict, inplace=True)


    # add additional columns as required by MUSE
    comsuption_df['RegionName'] = region # 'UK'
    #comsuption_df['ProcessName'] = 'service' # deprecated from MUSE 
    comsuption_df['Timeslice'] = list(range(1, len(comsuption_df.index) + 1))


    # Define the desired column order
    column_order = ['RegionName', 'Timeslice'] + [col for col in comsuption_df.columns if col not in ['RegionName', 'Timeslice']]
    # Reorder the DataFrame columns
    comsuption_df = comsuption_df[column_order]


    # Save the DataFrame to a CSV file
    #comsuption_df.to_csv(output_folder / f'Consumption{year}.csv', index=False)


comsuption_df.head()

,RegionName,Timeslice,RES.COOKING.OTHER,RES.COOKING.HOBS,RES.COOKING.OVENS,RES.CONSUMER-ELECTRONICS.TV,RES.FREEZERS,RES.COOLING,RES.COMPUTERS,RES.REFRIGERATORS,RES.LIGHTING,RES.OTHER,RES.WET.APPLIANCES,RES.SPACE-HEAT.EXISTING-AVERAGE,RES.SPACE-HEAT.NEW-AVERAGE,RES.HOT-WATER.EXISTING-AVERAGE,RES.HOT-WATER.NEW-AVERAGE
0,UK,1,3.341244,4.558006,4.102205,7.025666,2.075815,0.0,2.009059,4.749480,23.363731,3.299248,6.242203,82.547749,17.046507,21.011374,8.302680
1,UK,2,0.668249,0.911601,0.820441,2.810266,0.593090,0.0,1.095850,1.356994,93.454924,1.319699,2.496881,27.453535,5.669287,4.152715,1.640952
2,UK,3,0.334124,0.455801,0.410221,1.756417,1.037907,0.0,0.292227,2.374740,23.363731,1.154737,0.000000,31.364781,6.476978,13.923729,5.501985
3,UK,4,2.338871,3.190604,2.871544,2.458983,0.444817,0.0,0.986265,1.017746,93.454924,0.989774,1.872661,45.866424,9.471637,10.116177,3.997424
4,UK,5,3.341244,4.558006,4.102205,7.025666,2.075815,0.0,2.009059,4.749480,23.363731,3.299248,6.242203,102.662169,21.200231,21.152284,8.358360


## Create 2."Technodata.csv"

In [10]:
# Read data from the TIMES data in to dataframes
data_file = 'Residential sector - process techno-economic parameters.xlsx'
TechnoData = pd.read_excel(data_folder / data_file, skiprows=[0])

In [11]:
# All the process/technology names
TechList_all = TechnoData['process'].unique()

# For this analysis, we do not need technologies that are related to district heating or infrastructure.
# Define the substrings to filter out.
tech_remove = [".DISTN", "DISTRICT", "INFRASTRUCTURE","UK-RESOURCE"]
# Use a list comprehension to filter out strings containing any of the substrings
TechList = [s for s in TechList_all if not any(sub in s for sub in tech_remove)]

# print(len(TechList))

In [12]:
## need  to manually assign the fuel type for some technologies.
fuel_mapping = {
    'ELC': ['COMPUTERS', 'AIR-CONDITIONER', 'FREEZERS', 'REFRIGERATORS', 'LIGHTING', 'WET-APPLIANCES', 'CONSUMER-ELECTRONICS', 'ELECTRIC','ELECTRICITY','.KETTLE.','00.ELC','.01.ELC.'],
    'NGA': ['.GAS.','.NATURAL-GAS','.NGA.'],
    'OIL': ['.OIL'],
    'HCO': ['.HCO'],
    'WOOD': ['.WOD'],
    'LTH': ['.LTH'],
    'COAL': ['.COL','COAL'],
    'HYDROGEN': ['.HYDROGEN','.HDG.'],
    'KEROSENE': ['.KEROSENE','.KER.'],
    'LOGS': ['.LOGS'],
    'LFO': ['LIGHT-FUEL-OIL','.LFO'],
    'LPG': ['LIQUEFIED-PETROLEUM-GAS','.LPG'],
    'NGA-BOM-HYG': ['.NGA-BOM-HYG'],
    'COKE': ['.COKE'],
    'PELLETS': ['.PELLETS','.PEL'],
    'SOLAR': ['.SOL.', '.SOLAR.'],
}

#### extract data from TIMES data set

In [13]:
# create a dictionary to store the techno-economic data for each technology
Technodata_all = {}


for tech in TechList:
    # print('\n',tech)

    # Create a dictionary to store the technology data
    techdata = {}

    # Add the technology name to the dictionary
    techdata['ProcessName'] = tech
    ##=============the cap_par parameter=================##
    # Filtering the dataframe where 'process' is tech and 'attribute' is a specific value [NCAP_COST]
    cap_par = TechnoData.loc[
        (TechnoData['process']== tech) & 
        (TechnoData['attribute'] == "NCAP_COST [Investment cost per unit of new capacity installed]"), 
        [base_year]
    ]
    # print(cap_par)

        # If no rows are found, assign 'NA'
    if cap_par.empty:
        techdata['cap_par'] = np.nan
    else:
        techdata['cap_par'] = cap_par[base_year].values[0]  # Extract the value

    

    ##===================the fix_par parameter===================##
    fix_par =  TechnoData.loc[
        (TechnoData['process']== tech) & 
        (TechnoData['attribute'] == "NCAP_FOM [Fixed operating and maintenance cost per unit of capacity according to the year initially installed.]"), 
        [base_year]
    ]

    # If no rows are found, assign 'NA'
    if fix_par.empty:
        techdata['fix_par'] = np.nan
    else:
        techdata['fix_par'] = fix_par[base_year].values[0]  # Extract the value



    ##===================the TechnicalLife parameter===================##
    TechnicalLife =  TechnoData.loc[(TechnoData['process']== tech) & 
        (TechnoData['attribute'] == "NCAP_TLIFE [Technical life-time of a Process;number of years.Default:G_TLIFE.]"), 
        [base_year]
    ]
    # If no rows are found, assign 'NA'
    if TechnicalLife.empty:
        techdata['TechnicalLife'] = np.nan
    else:
        techdata['TechnicalLife'] = TechnicalLife[base_year].values[0]  # Extract the value

    # print(techdata['TechnicalLife'])    


    ##===================the efficiency parameter===================##
    efficiency =  TechnoData.loc[(TechnoData['process']== tech) & 
        (TechnoData['attribute'] == "ACT_EFF [Generic process transformation parameter]"), 
        [base_year]
    ]
    # If no rows are found, assign 'NA'
    if efficiency.empty:
        techdata['efficiency'] = np.nan
    else:
        techdata['efficiency'] = efficiency[base_year].values[0]  # Extract the value

    #print(techdata['efficiency'])    
    

    ##===================the UtilizationFactor parameter===================##
    UtilizationFactor =  TechnoData.loc[
        (TechnoData['process'] == tech) & 
        (
            (TechnoData['attribute'] == "NCAP_AFA [Annual Availability factor relating a unit of production to the installed capacity according to the year initial installed.Fraction]") | 
            (TechnoData['attribute'] == "NCAP_AFC [Commodity-specific availability factor]")
        ), 
        [base_year]
    ]

    if UtilizationFactor.empty:
        techdata['UtilizationFactor'] = 1 # This will be overwritten by "TechnodataTimeslice.csv"
    else:
        techdata['UtilizationFactor'] = UtilizationFactor[base_year].values[0]  # Extract the value
    
    
    ##===================the Total Capacity Limit===================##
    # "TotalCapacityLimit" in MUSE

    TotalCapacityLimit =  TechnoData.loc[
        (TechnoData['process'] == tech) & 
        (TechnoData['attribute'] == "CAP_BND [Bound on the total installed capacity  in a time period]") , 
        [base_year]
    ]

    if TotalCapacityLimit.empty:
        techdata['TotalCapacityLimit'] = 100000000  # Default value
    else:
        techdata['TotalCapacityLimit'] = TotalCapacityLimit[base_year].values[0]  # Extract the value


    ##===================Max Capacity Addition===================##
    # "MaxCapacityAddition" in MUSE
    MaxCapacityAddition =  TechnoData.loc[
        (TechnoData['process'] == tech) & 
        (TechnoData['attribute'] == "NCAP_BND [Limit on investments in new capacity,Ignored if y-index prior to 1st MODLYEAR]") , 
        [0] # I don't know why the column name is called "0"
    ]

    if MaxCapacityAddition.empty:
        techdata['MaxCapacityAddition'] = 100000000  # Default value
    else:
        techdata['MaxCapacityAddition'] = MaxCapacityAddition[0].values[0]


    ##===================the Fuel parameter===================##
    # "Fuel" in MUSE
    techdata['Fuel'] = np.nan  # Default value

    for fuel, keywords in fuel_mapping.items():
        if any(keyword in techdata['ProcessName'] for keyword in keywords):
            techdata['Fuel'] = fuel
            break
    else:
        # Extract the matching text
        fuel_match = re.search(r"\.([^.]*)\.\s*$", tech)
        # Check if a match was found and print the result
        if fuel_match:
            techdata['Fuel'] = fuel_match.group(1)


    ##===================the Type parameter===================##
    ### skip the "Type" for now, as it is an optional parameter in MUSE
    # "Type" in MUSE
    # type_match = re.search(r": \.(.*)\.?$", tech)
    # if type_match:
    #     techdata['Type'] = type_match.group(1)
    # else:
    #     techdata['Type'] = np.nan
        
    
    # add the technology data to the Master dictionary
    Technodata_all[tech] = techdata
    # break

# Create a DataFrame from the dictionary of dictionaries
Technodata_df = pd.DataFrame.from_dict(Technodata_all, orient='index')

Technodata_df.head()


,ProcessName,cap_par,fix_par,TechnicalLife,efficiency,UtilizationFactor,TotalCapacityLimit,MaxCapacityAddition,Fuel
RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICROWAVE],RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,NaN,0.000000,5.0,0.800000,1.0,100000000.0,2.0,ELC
RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],NaN,8.232143,14.0,1.096566,1.0,100000000.0,2.0,ELC
RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],NaN,8.232143,14.0,1.518031,1.0,100000000.0,2.0,ELC
RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],NaN,8.785714,14.0,0.800000,1.0,100000000.0,2.0,NGA
RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],NaN,8.785714,14.0,1.115535,1.0,100000000.0,2.0,NGA


In [14]:
# Add data for the 'EndUse' column:
## We will generate a process's 'EndUse' by looking at ProcessName and comparing it with the 'commodity_df' column in the Consumption.csv, and use the "get_close_matches" to name it.

services = [item for item in comsuption_df.columns if '.' in item] # get all the services name and remove the 'RegionName' and 'Timeslice'. 

processes = Technodata_df['ProcessName'].to_list() # get all the processes/technologies name

# first, create the dictionary with closest matches
service_to_enduse = {}
for process in processes:
    proc = re.search(r"\[(.*?)\]", process).group(1)
    # print(proc)

    if 'AIR-CONDITIONER' in proc: # need to manually assign the service for this process
        service_to_enduse[process] = 'RES.COOLING'
        continue
    elif 'CHP' in proc: # the "get_close_matches" functiion will assign it to 'RES.HOT-WATER.NEW-AVERAGE'
        service_to_enduse[process] = 'RES.SPACE-HEAT.EXISTING-AVERAGE'
        continue
    else:
        # Get the closest matching service for the current process item
        closest_match = get_close_matches(proc, services, n=1)
        

        if not closest_match: # if no match is found, consider only the base part of the process
            if ':' in proc:
                proc_base = proc.split(":")[0]  # Consider only the base part before ':'
            else:
                proc_base = re.split(r'\d+', proc)[0]  # Consider only the base part before any number
            closest_match = get_close_matches(proc_base, services, n=1)
        
        # Assign the service with the closest match to the process
        if closest_match:
            service_name = closest_match[0]
        else:
            service_name = None  # or some default value if no match is found

        service_to_enduse[process] = service_name
    # print(service_to_enduse[process])

    # break

#Then, apply the mapping to create the 'EndUse' column
Technodata_df['EndUse'] = Technodata_df['ProcessName'].map(service_to_enduse)

### Fill in missing values

In [15]:
# There are missing values in the "cap_par", "fix_par", "TechnicalLife", "EndUse" ect. columns.
# Define a generalized function that can fill missing values for any specified column based on similar "ProcessName"
def fill_missing_column_value(row, data, target_column):
    """
    This function takes a row, the complete data frame, and a target column name.
    It finds similar processes (based on ProcessName) with non-missing values in the target column
    and returns the value from the most similar process.
    """
    # Extract the process name for the current row
    process_name = row['ProcessName']
    
    # Filter out rows with non-missing values in the target column
    available_data = data.dropna(subset=[target_column])
    
    # Find similar process names using the difflib's get_close_matches function
    similar_names = get_close_matches(process_name, available_data['ProcessName'], n=1, cutoff=0.5)
    
    # If a similar name is found, return the corresponding value in the target column
    if similar_names:
        similar_row = available_data[available_data['ProcessName'] == similar_names[0]]
        return similar_row[target_column].values[0]
    return None

In [16]:
# Apply the function to fill missing values in "cap_par", "fix_par"
for column in ['cap_par', 'fix_par', 'TechnicalLife','EndUse']:
    Technodata_df[column] = Technodata_df.apply(lambda row: fill_missing_column_value(row, Technodata_df, column) 
                              if pd.isna(row[column]) else row[column], axis=1)
    

# Apply 'fillnan()' to the "Fuel" column:
Technodata_df['Fuel'] = Technodata_df['Fuel'].fillna('ELC') 

In [17]:
# further check for missing values (optional)
print(Technodata_df.isnull().any())

# check for missing values (in which rows)
print(Technodata_df[Technodata_df.isnull().any(axis=1)])


ProcessName            False
cap_par                False
fix_par                False
TechnicalLife          False
efficiency             False
UtilizationFactor      False
TotalCapacityLimit     False
MaxCapacityAddition    False
Fuel                   False
EndUse                 False
dtype: bool
Empty DataFrame
Columns: [ProcessName, cap_par, fix_par, TechnicalLife, efficiency, UtilizationFactor, TotalCapacityLimit, MaxCapacityAddition, Fuel, EndUse]
Index: []


In [18]:
# add other columns that is required by MUSE
Technodata_df['RegionName'] = region
Technodata_df['Time'] = base_year
Technodata_df['cap_exp'] = 1
Technodata_df['fix_exp'] = 1
Technodata_df['var_exp'] = 1
Technodata_df['var_par'] = 0
Technodata_df['MaxCapacityGrowth'] = 100 #%
Technodata_df['ScalingSize'] = 1
Technodata_df['InterestRate'] = 0.1
Technodata_df['Agent1'] = 1 # for now, we create a single agent. Ofgem agents will be added later/different version.


In [19]:
# Add the "Unit" row to the DataFrame
# Step 1: Create the dictionary with the corresponding unit values
unit_row = {
    "ProcessName" : "Unit",
    "RegionName": "-",
    "Time" : "year",
    "cap_par" : "MGBP2020/PJ_a",
    "cap_exp" : "-",
    "fix_par" : "MGBP2020/PJ_a",
    "fix_exp" : "-",
    "var_par" : "MGBP2020/PJ",
    "var_exp" : "-",
    "MaxCapacityAddition" : "PJ",
    "MaxCapacityGrowth" : "%",
    "TotalCapacityLimit" : "PJ",
    "TechnicalLife" : "years",
    "UtilizationFactor" : "-",
    "ScalingSize" : "PJ",
    "efficiency" : "%",
    "InterestRate" : "-",
    #"Type" : "-", 
    "Fuel" : "-", 
    "EndUse" : "-", 
    "Agent1" : "new" 
}

# Convert the dictionary into a DataFrame
unit_df = pd.DataFrame([unit_row])

# Step 3: Concatenate the unit row DataFrame on top of Technodata_final
Technodata_final = pd.concat([unit_df, Technodata_df]).reset_index(drop=True)


# Rearrange the columns to ensure 'ProcessName' is the first column
columns = ['ProcessName'] + [col for col in Technodata_final.columns if col != 'ProcessName']
Technodata_final = Technodata_final[columns]

Technodata_final

,ProcessName,RegionName,Time,cap_par,cap_exp,fix_par,fix_exp,var_par,var_exp,MaxCapacityAddition,MaxCapacityGrowth,TotalCapacityLimit,TechnicalLife,UtilizationFactor,ScalingSize,efficiency,InterestRate,Fuel,EndUse,Agent1
0,Unit,-,year,MGBP2020/PJ_a,-,MGBP2020/PJ_a,-,MGBP2020/PJ,-,PJ,%,PJ,years,-,PJ,%,-,-,-,new
1,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,UK,2010,61.17761,1,0.0,1,0,1,2.0,100,100000000.0,5.0,1.0,1,0.8,0.1,ELC,RES.COOKING.OTHER,1
2,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],UK,2010,164.642857,1,8.232143,1,0,1,2.0,100,100000000.0,14.0,1.0,1,1.096566,0.1,ELC,RES.COOKING.HOBS,1
3,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],UK,2010,164.642857,1,8.232143,1,0,1,2.0,100,100000000.0,14.0,1.0,1,1.518031,0.1,ELC,RES.COOKING.OVENS,1
4,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],UK,2010,175.714286,1,8.785714,1,0,1,2.0,100,100000000.0,14.0,1.0,1,0.8,0.1,NGA,RES.COOKING.HOBS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,RWEAWHTRE01 [RES.WATER.EXISTING-AVERAGE: .01.E...,UK,2010,2.621435,1,0.262143,1,0,1,100000000.0,100,100000000.0,15.0,1.0,1,0.85,0.1,ELC,RES.HOT-WATER.EXISTING-AVERAGE,1
160,RWEAWHTRG01 [RES.WATER.EXISTING-AVERAGE: .01.N...,UK,2010,6.108552,1,0.610855,1,0,1,100000000.0,100,100000000.0,15.0,1.0,1,0.8,0.1,NGA,RES.HOT-WATER.EXISTING-AVERAGE,1
161,RWNASA01 [RES.WATER.NEW-AVERAGE: .01.STANDALON...,UK,2010,0.000593,1,0.000593,1,0,1,100000000.0,100,100000000.0,50.0,1.0,1,1.0,0.1,ELC,RES.HOT-WATER.NEW-AVERAGE,1
162,RWNAWHTRE01 [RES.WATER.NEW-AVERAGE: .01.ELC.RE...,UK,2010,2.097148,1,0.209715,1,0,1,100000000.0,100,100000000.0,15.0,1.0,1,0.85,0.1,ELC,RES.HOT-WATER.NEW-AVERAGE,1


In [20]:
# Save the the df as 'Technodata.csv'. 
#Technodata_final.to_csv(output_folder / 'Technodata.csv', index=False)


## Create 3. 'Technodata_Timeslice.csv'

In [21]:
# Sample data for seasons and periods
seasons = ['A', 'P', 'S', 'W']  # Four seasons: A (Autumn), P (Spring), S (Summer), W (Winter)
periods = ['Day', 'Evening', 'Night', 'Peak']  # Originally was called: Day, Late_evening, Night, Evening_peak, but does not match with the TIMES data
# seasons_periods = ["AD","AE","AN","AP","PD","PE","PN","PP","SD","SE","SN","SP","WD","WE","WN","WP" ] # 16 time slices in TIMES model

#### generate the 'UtilizationFactor' data for each technology at each time slices

In [22]:
# List to store rows
data = []

# Loop over all the technologies and generate rows for each one
for tech in Technodata_final['ProcessName'][1:].tolist(): # Skip the first row (Unit)
    for season in seasons:
        for period in periods:
            result = TechnoData.loc[
                (TechnoData['process'] == tech) & 
                (TechnoData['time_slice'] == str(season+period[0])) &
                (
                    (TechnoData['attribute'] == "NCAP_AF [Availability factor relating a unit of production to the installed capacity according to the year initial installed.]") | 
                    (TechnoData['attribute'] == "FLO_FR [Load curve describing the availability of a commodity in time-slices of the period;fraction.Default:none[MM-SRCENCP FR,but for any process/seasonal commodity not just SRCENCP/electricity]]")
                ), 
                ['lim_type', base_year]
            ]
            # print(result)

            # Assign MinimumServiceFactor with a default value of 0 if not found
            if 'LO' in result['lim_type'].values: # Lower bound
                MinimumServiceFactor = result[result['lim_type'] == 'LO'][base_year].values[0]
            else:
                MinimumServiceFactor = 0

            # Assign UtilizationFactor with a default value of 1 if not found
            if 'UP' in result['lim_type'].values: # Upper bound
                utilization_factor = result[result['lim_type'] == 'UP'][base_year].values[0]
            else:
                utilization_factor = 1

            # Time column is the year
            time = base_year

            # region
            RegionName = region

            # Append each row to the data list
            data.append([tech, RegionName, time, season, period, utilization_factor, MinimumServiceFactor])

# Create a Pandas DataFrame
Timeslice_df = pd.DataFrame(data, columns=['ProcessName', 'RegionName', 'Time', 'season', 'period', 'UtilizationFactor','MinimumServiceFactor'])

# Save the DataFrame to a CSV file
#Timeslice_df.to_csv(output_folder / 'TechnodataTimeslices.csv', index=False, float_format='%.6f', encoding='utf-8')

print('{TechnodataTimeslices.csv} created successfully.')

{TechnodataTimeslices.csv} created successfully.


## Create 4. "GlobalCommodities.csv" 

#### We need the list of technologies from 'Technodata.csv' . This is used for extracting the "Fuel" used in the technologies.
#### Or this can be obtained from the "Technodata_final" dataframe, which we created earlier (we will use this approach here)

In [23]:
# Fuel_EndUse = Technodata_final.iloc[1:][['Fuel', 'EndUse']].copy()

fuels = Technodata_final.iloc[1:]['Fuel'].unique().tolist() #  get the 'fuels' from the "Fuel" column

services = [col for col in comsuption_df.columns if col not in ["RegionName", "Timeslice"]] # Get the name of the services from the column names in the comsuption_df

In [24]:
# Global commodities will be consistuted of the unique values of 'Fuel' column and 'services'.
energy_commodity = pd.DataFrame({'Commodity': fuels, 'CommodityType': 'Energy'}) # we name it as energy_commodity as MUSE uses this name.

service_commodity = pd.DataFrame({'Commodity': services, 'CommodityType': 'Services'}) # we name it as energy_commodity as MUSE uses this name.

In [25]:
# Concatenate both DataFrames to form the final result
GlobalCommodities_df = pd.concat([energy_commodity, service_commodity], ignore_index=True)

# add the CO2f to the Commodity
GlobalCommodities_df.loc[len(GlobalCommodities_df)] = ['CO2f', 'Environmental']

# Add a new column 'CommodityName' with the same values as 'Commodity'
GlobalCommodities_df['CommodityName'] = GlobalCommodities_df['Commodity']


# save the new DataFrame to a new csv file
#GlobalCommodities_df.to_csv(output_folder / 'GlobalCommodities.csv', index=False)

#print('{GlobalCommodities_df.csv} created successfully.')

# display the df
# GlobalCommodities_df

## Create 5. CommIn.csv and CommOut.csv

In [28]:
from EmissionData import GHG_fuels#  need to use the  emission factor from the fossil_fuels, which is not given in the TIMES data set.
GHG_fuels

{'COKE': 107.73836722774595,
 'COAL': 90.98096129314096,
 'KEROSENE': 69.37073192046033,
 'OIL': 70.27115814432203,
 'LFO': 70.27115814432203,
 'HCO': 70.27115814432203,
 'LPG': 59.8072576059714,
 'InorganicWaste': 33.6948971139823,
 'NGA': 50.14900158370755}

In [29]:
# Create a new DataFrame with ProcessName as the first column and fuel types as the other columns
df_CommIn = pd.DataFrame(columns=['ProcessName'] + fuels)
#  Create a new DataFrame with ProcessName as the first column and service as the other columns, and an additional column for CO2 emissions
df_CommOut = pd.DataFrame(columns=['ProcessName'] + services + ['CO2f'])

# # Step 3: Populate the df_CommIn and df_CommOut DataFrame
for _, row in Technodata_final.iterrows():
    if row['ProcessName'] == 'Unit': # Skip the 'Unit' row
        continue
    else:
        # print(row)
        process_name = row['ProcessName']
        fuel_type = row['Fuel']
        efficiency = float(row['efficiency'])
        end_use = row['EndUse']
        
        ## ==== CommIn ==== ##
        # Create a new row as a DataFrame with 0s for all fuel types
        new_row1 = pd.DataFrame([[process_name] + [0]*len(fuels)], columns=['ProcessName'] + fuels)
        new_row1[fuel_type] = 1 / efficiency

        if df_CommIn.empty:
        # If df_CommIn is empty, initialize it with new_row1 directly
            df_CommIn = new_row1
        else:
            # Otherwise, concatenate as usual
            df_CommIn = pd.concat([df_CommIn, new_row1])


        ## ==== CommOut ==== ##
        # Create a new row as a DataFrame with 0s for all end_use types
        new_row2 = pd.DataFrame([[process_name] + [0]*len(services)+ [0]], columns=['ProcessName'] + services + ['CO2f'])
        new_row2[end_use] = 1 # set the value to 1
        
        if fuel_type in GHG_fuels.keys():
            new_row2['CO2f'] = GHG_fuels[fuel_type]
        
        if df_CommOut.empty:
        # If df_CommIn is empty, initialize it with new_row1 directly
            df_CommOut = new_row2
        else:
            # Otherwise, concatenate as usual
            df_CommOut = pd.concat([df_CommOut, new_row2])


print(df_CommOut)

                                          ProcessName  RES.COOKING.OTHER  \
0   RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...                  1   
0          RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.]                  0   
0        RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.]                  0   
0               RCHGHOB00 [RES.COOKING: .00.HOB.GAS.]                  0   
0             RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.]                  0   
..                                                ...                ...   
0   RWEAWHTRE01 [RES.WATER.EXISTING-AVERAGE: .01.E...                  0   
0   RWEAWHTRG01 [RES.WATER.EXISTING-AVERAGE: .01.N...                  0   
0   RWNASA01 [RES.WATER.NEW-AVERAGE: .01.STANDALON...                  0   
0   RWNAWHTRE01 [RES.WATER.NEW-AVERAGE: .01.ELC.RE...                  0   
0   RWNAWHTRG01 [RES.WATER.NEW-AVERAGE: .01.NGA.BO...                  0   

    RES.COOKING.HOBS  RES.COOKING.OVENS  RES.CONSUMER-ELECTRONICS.TV  \
0              

In [30]:
# add 'Level' to the df_CommIn as required by MUSE
df_CommIn['Level'] = 'fixed'  # 'fixed' or 'flexible'


for df in [df_CommIn, df_CommOut]:
    # add the RegionName as MUSE requires it.
    df['RegionName'] = region

    #add Time column as MUSE requires it.
    df['Time'] = base_year

    # add the 'Unit' row. Define the new row values based on the column names
    new_row = ["Unit"] + [
        "year" if col == "Time" 
        else "kt/PJ" if col == "CO2f" 
        else "-" if col in ["RegionName", "Level"]
        else "PJ/PJ"
        for col in df.columns[1:]
    ]

    # Add the unit row (as the last row) to the DataFrame
    df.loc[-1] = new_row


In [31]:
# save the data to csv files
df_CommIn.to_csv(output_folder / 'CommIn.csv', index=False)
df_CommOut.to_csv(output_folder / 'CommOut.csv', index=False)

## Create the 6. "Projections.csv"

In [40]:
from PriceProjection import price_projections
price_projections

{'COKE': {'2010': 14.033,
  '2015': 13.5745,
  '2020': 13.684,
  '2025': 13.684,
  '2030': 13.684,
  '2035': 13.684,
  '2040': 13.684,
  '2045': 13.684,
  '2050': 13.684},
 'COAL': {'2010': 9.83,
  '2015': 10.265,
  '2020': 10.867488,
  '2025': 10.867488,
  '2030': 10.867488,
  '2035': 10.867488,
  '2040': 10.867488,
  '2045': 10.867488,
  '2050': 10.867488},
 'KEROSENE': {'2010': 18.21616657,
  '2015': 13.98537431,
  '2020': 16.6741146,
  '2025': 16.6741146,
  '2030': 16.6741146,
  '2035': 16.6741146,
  '2040': 16.6741146,
  '2045': 16.6741146,
  '2050': 16.6741146},
 'LFO': {'2010': 19.7944343,
  '2015': 17.9035,
  '2020': 16.748,
  '2025': 16.748,
  '2030': 16.748,
  '2035': 16.748,
  '2040': 16.748,
  '2045': 16.748,
  '2050': 16.748},
 'HCO': {'2010': 19.7944343,
  '2015': 17.9035,
  '2020': 16.748,
  '2025': 16.748,
  '2030': 16.748,
  '2035': 16.748,
  '2040': 16.748,
  '2045': 16.748,
  '2050': 16.748},
 'LPG': {'2010': 13.476,
  '2015': 15.779,
  '2020': 16.709824,
  '2025': 1

In [ ]:
# set time range that we are interested in
years = [str(year) for year in range(2010, 2050+1, 5)] #[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045]

# Initialize an empty dictionary to store projections
projections = {}

# Loop over the commodities and populate the projections dictionary
for comm in GlobalCommodities_df['CommodityName'].to_list():
    if comm in price_projections.keys():
        # If the commodity exists in price_projections, add its data
        projections[comm] = price_projections[comm]
    else:
        # If the commodity is not in price_projections, initialize with years as keys and 0 as values
        projections[comm] = {year: 0 for year in years}

# Convert the projections dictionary to a DataFrame
proj_df = pd.DataFrame(projections)#.reset_index(drop=True)

# Reset the index to include the dictionary keys (years) as the first column
proj_df = proj_df.rename_axis("Time").reset_index()

# Add the 'RegionName' and 'Attribute' columns as required by the MUSE format
proj_df.insert(0, 'RegionName', region)  # Insert 'Region' as the first column
proj_df.insert(1, 'Attribute', 'CommodityPrice')  # Insert 'region' as the first column


,RegionName,Attribute,Time,ELC,NGA,OIL,HCO,WOOD,SOLAR,HYDROGEN,...,RES.COMPUTERS,RES.REFRIGERATORS,RES.LIGHTING,RES.OTHER,RES.WET.APPLIANCES,RES.SPACE-HEAT.EXISTING-AVERAGE,RES.SPACE-HEAT.NEW-AVERAGE,RES.HOT-WATER.EXISTING-AVERAGE,RES.HOT-WATER.NEW-AVERAGE,CO2f
0,UK,CommodityPrice,2010,43.342675,12.552000,19.794434,19.794434,10.019000,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
1,UK,CommodityPrice,2015,45.822469,12.479500,19.794434,17.903500,11.848500,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
2,UK,CommodityPrice,2020,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
3,UK,CommodityPrice,2025,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
4,UK,CommodityPrice,2030,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
5,UK,CommodityPrice,2035,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
6,UK,CommodityPrice,2040,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
7,UK,CommodityPrice,2045,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
8,UK,CommodityPrice,2050,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# save the new DataFrame to a new csv file
proj_df.to_csv(output_folder / 'Projections.csv', index=False)

# display the df
proj_df

,RegionName,Attribute,Time,ELC,NGA,OIL,HCO,WOOD,SOLAR,HYDROGEN,...,RES.COMPUTERS,RES.REFRIGERATORS,RES.LIGHTING,RES.OTHER,RES.WET.APPLIANCES,RES.SPACE-HEAT.EXISTING-AVERAGE,RES.SPACE-HEAT.NEW-AVERAGE,RES.HOT-WATER.EXISTING-AVERAGE,RES.HOT-WATER.NEW-AVERAGE,CO2f
0,UK,CommodityPrice,2010,43.342675,12.552000,19.794434,19.794434,10.019000,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
1,UK,CommodityPrice,2015,45.822469,12.479500,19.794434,17.903500,11.848500,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
2,UK,CommodityPrice,2020,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
3,UK,CommodityPrice,2025,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
4,UK,CommodityPrice,2030,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
5,UK,CommodityPrice,2035,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
6,UK,CommodityPrice,2040,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
7,UK,CommodityPrice,2045,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0
8,UK,CommodityPrice,2050,57.166667,11.083333,19.794434,16.748000,12.692744,0.0,105.016835,...,0,0,0,0,0,0,0,0,0,0


## Create 7."ExistingCapacity.csv"

#### Step 1. Read-in the "PRC_RESID [Residual Capacity]" parameters in the TIMES data set.

In [59]:
#### Read-in the list of technologies that will be used 
data_file = 'Residential sector - process techno-economic parameters.xlsx'
# file_path_full = os.path.abspath(os.path.join(file_path, "..", data_folder, data_file1))

# Read the data into a DataFrame
tech_econ_data = pd.read_excel( data_folder / data_file, skiprows=[0])

# Display the first few rows of the DataFrame
tech_econ_data.head()


,side,lim_type,time_slice,attribute,user_constraint,commodity,commodity_group,process,scenario,Unnamed: 9,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,-,-,-,END [-],-,-,-,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,BASE,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-,-,-,END [-],-,-,-,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],BASE,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-,-,-,END [-],-,-,-,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],BASE,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-,-,-,END [-],-,-,-,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],BASE,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-,-,-,END [-],-,-,-,RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],BASE,2010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
Residual_Capacity = tech_econ_data[tech_econ_data['attribute'] == "PRC_RESID [Residual Capacity]"].copy()

# Using DataFrame.drop to remove the columns that are not needed
Residual_Capacity.drop(columns=['side','lim_type','time_slice','user_constraint','commodity','commodity_group','scenario','Unnamed: 9',0], inplace=True)

# Display the first few rows of the DataFrame
Residual_Capacity.head()


,attribute,process,2010,2011,2015,2016,2017,2020,2023,2024,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
1105,PRC_RESID [Residual Capacity],RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,55.418544,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1106,PRC_RESID [Residual Capacity],RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],33.820706,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1107,PRC_RESID [Residual Capacity],RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],47.384432,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108,PRC_RESID [Residual Capacity],RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],41.779294,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1109,PRC_RESID [Residual Capacity],RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],20.655568,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Define a function to explicitly calculate linear interpolation between start (year) and cutoff years (the year where the residual capacity reaches 0)
# because the data in TIMES data set is not complete. As you can see in the above table, the residual capacity is not available for all years (as indicated by NaN values).

def explicit_interpolation(row):
    # Identify year columns and convert them to numeric
    year_columns = [col for col in Residual_Capacity.columns if isinstance(col, int)]
    row[year_columns] = row[year_columns].apply(pd.to_numeric, errors='coerce')
    
    # Starting point
    start_year = year_columns[0]
    start_value = row[start_year]
    
    # Identify cutoff year
    cutoff_year = None
    for year in year_columns:
        if row[year] == 0:
            cutoff_year = year
            break
    
    # Interpolation between start and cutoff
    if cutoff_year:
        start_idx = year_columns.index(start_year)
        cutoff_idx = year_columns.index(cutoff_year)
        
        # Calculate interpolated values
        for i in range(start_idx + 1, cutoff_idx):
            year = year_columns[i]
            row[year] = start_value + (row[cutoff_year] - start_value) * (i - start_idx) / (cutoff_idx - start_idx)
        
        # Set values after cutoff to 0
        for year in year_columns[cutoff_idx + 1:]:
            row[year] = 0
    else:
        # If no cutoff, set all years after start to 0
        row.loc[year_columns[1:]] = 0

    return row

In [62]:
# Apply this interpolation method
Residual_Capacity_interpolation = Residual_Capacity.apply(explicit_interpolation, axis=1)

# List of columns to keep
years = list(range(2010,2050+1,5)) # years of interest
columns_to_keep = ["process"] + years

# Select the ones you want
Residual_Capacity_interpolation = Residual_Capacity_interpolation[columns_to_keep].reset_index(drop=True)

# In order to further processing(concatenating) in later step 3, we need to rephrase the name in the "process" column
# Residual_Capacity_interpolation['process'] = Residual_Capacity_interpolation['process'].str.extract(r'\[(.*?)\]')[0] # Extract the text between "[" and "]"


Residual_Capacity_interpolation.head()

,process,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,55.418544,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],33.820706,24.157647,9.663059,0.0,0.0,0.0,0.0,0.0,0.0
2,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],47.384432,33.846023,13.538409,0.0,0.0,0.0,0.0,0.0,0.0
3,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],41.779294,29.842353,11.936941,0.0,0.0,0.0,0.0,0.0,0.0
4,RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],20.655568,14.753977,5.901591,0.0,0.0,0.0,0.0,0.0,0.0


#### Step 2. Then Read-in the list of technologies that will be used 
##### * We can retrive this information from the dataframe 'Technodata_final' that we created earlier in this Jupter Notebook or directly from 'Technodata.csv'
##### * Here we read directly from 'Technodata.csv'

In [63]:
# make sure you have created 'Technodata.csv' already.
data_file = 'Technodata.csv'

# read the csv file, we only need the column 'ProcessName'
df_tech_lst = pd.read_csv(output_folder / data_file, usecols=['ProcessName'],skiprows=[1]) # skip the 'Unit' row

df_tech_lst.head()

,ProcessName
0,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...
1,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.]
2,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.]
3,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.]
4,RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.]


#### Step 3. Concatenate/Merge the above two dataframes ( 'Residual_Capacity_interpolation' and 'df_tech_lst')

In [64]:
# merge the two DataFrames, while keeping all the rows in the left DataFrame (i.e. df_tech_lst)
df_merged = df_tech_lst.merge(Residual_Capacity_interpolation, left_on='ProcessName', right_on='process', how='left')

# Replace NaN values in year columns with 0 for non-matching rows
df_merged[years] = df_merged[years].fillna(0)

# drop the "Process" column
df_merged.drop('process', axis=1, inplace=True)


#### Step 4.  Format the merged df according to MUSE requirement and save it as a .csv file

In [65]:
# set the region and unit.
# region = 'UK'
unit = 'PJ/y'


# rename the columns
df_merged.rename(columns={'MemberDesc': 'ProcessName'}, inplace=True)

# add the region and unit columns
df_merged['RegionName'] = region
df_merged['Unit'] = unit

# save the DataFrame to a csv file
df_merged.to_csv(output_folder / 'ExistingCapacity.csv', index=False)

# display the DataFrame
df_merged.head()



,ProcessName,2010,2015,2020,2025,2030,2035,2040,2045,2050,RegionName,Unit
0,RCEOTHER00 [RES.COOKING: .00.OTHER.KETTLE.MICR...,55.418544,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,UK,PJ/y
1,RCHEHOB00 [RES.COOKING: .00.HOB.ELECTRIC.],33.820706,24.157647,9.663059,0.0,0.0,0.0,0.0,0.0,0.0,UK,PJ/y
2,RCHEOVEN00 [RES.COOKING: .00.OVEN.ELECTRIC.],47.384432,33.846023,13.538409,0.0,0.0,0.0,0.0,0.0,0.0,UK,PJ/y
3,RCHGHOB00 [RES.COOKING: .00.HOB.GAS.],41.779294,29.842353,11.936941,0.0,0.0,0.0,0.0,0.0,0.0,UK,PJ/y
4,RCHGOVEN00 [RES.COOKING: .00.OVEN.GAS.],20.655568,14.753977,5.901591,0.0,0.0,0.0,0.0,0.0,0.0,UK,PJ/y


## Create 8. "Agent.csv"
#### * For this version of the model, we create a simple representation of agents - 2 agents.
#### * In a different version of the input file, we will create more types of agents.

In [ ]:
# import Agent from another script
# import cls_Agent #(already imported above)
# importlib.reload(cls_Agent) 
# from cls_Agent import Agent

In [ ]:
# Convert the list of Agent instances to a list of dictionaries
data = [vars(instance) for instance in Agent.instances]

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

# Rename the 'AgentType' column to 'Type' to match the MUSE model requirements
df.rename(columns={'AgentType': 'Type'}, inplace=True) 


# Write the DataFrame to a CSV file
csv_file = output_folder / 'Agent.csv'
df.to_csv(csv_file, index=False, float_format='%.6f', encoding='utf-8')

print(f"CSV file '{csv_file}' created successfully.")